In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split
import plotly.express as px
import emd
tf.keras.mixed_precision.set_global_policy('mixed_float16')

BATCH_SIZE = 32
CHANNEL_NUMBER = 3
WINDOW_SIZE = 300
SLIDING_STEP = int(WINDOW_SIZE * 0.4)
CLASS_NUMBER = 2
if CLASS_NUMBER < 3:
    CLASS_NUMBER = 1

def slicingAndNormalize(arr):
    def normalizer(X):
        mean = np.average(X, axis=1)
        std = np.std(X, axis=1)
        ret = (X.copy() - mean) / std
        return ret
    
    totalLength = arr.shape[0]
    if totalLength <= WINDOW_SIZE:
        return arr
    ret = normalizer((arr[:WINDOW_SIZE, :])[np.newaxis, :])
    
    i = SLIDING_STEP
    while (totalLength - i) > WINDOW_SIZE:
        new = normalizer((arr[i:(i + WINDOW_SIZE), :])[np.newaxis, :])
        ret = np.concatenate([ret, new], axis=0)
        i += SLIDING_STEP
    return ret

def slicing(arr):
    totalLength = arr.shape[0]
    if totalLength <= WINDOW_SIZE:
        return arr
    ret = (arr[:WINDOW_SIZE, :])[np.newaxis, :]
    
    i = SLIDING_STEP
    while (totalLength - i) > WINDOW_SIZE:
        new = (arr[i:(i + WINDOW_SIZE), :])[np.newaxis, :]
        ret = np.concatenate([ret, new], axis=0)
        i += SLIDING_STEP
    return ret

def emdSignal(sig):
    dataNumber = sig.shape[0]
    channel = sig.shape[-1]
    ret = None
    
    for i in range(dataNumber):
        temp = None
        
        for c in range(channel):
            raw = sig[i, :, c]
            imf = emd.sift.sift(raw, max_imfs=4, imf_opts={'sd_thresh': 0.1})
            
            if imf.shape[-1] < 4:
                compensate = np.zeros((WINDOW_SIZE, 4 - imf.shape[-1]))
                imf = np.concatenate([imf, compensate], axis = 1)
            
            if not type(temp) == np.ndarray: 
                temp = imf
            else: 
                temp = np.concatenate([temp, imf], axis = 1)
            
        if type(temp) == np.ndarray: 
            if not type(ret) == np.ndarray: 
                ret = temp[np.newaxis, :]
            else: 
                ret = np.concatenate([ret, temp[np.newaxis, :]], axis = 0)
                
    return ret

def buildModel(shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape = shape),
        tf.keras.layers.Conv1D(80, 40, padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(axis=1),
        tf.keras.layers.MaxPool1D(padding='same'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Conv1D(100, 50, padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(axis=1),
        tf.keras.layers.MaxPool1D(padding='same'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Conv1D(120, 60, padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(axis=1),
        tf.keras.layers.MaxPool1D(padding='same'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(CLASS_NUMBER)]#, activation='softmax' if CLASS_NUMBER > 2 else 'sigmoid')]
    )
    model.compile(optimizer='adam',
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True) if CLASS_NUMBER > 2 else tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.CategoricalAccuracy(),
                         tf.keras.metrics.Precision(thresholds = 0.5),
                         tf.keras.metrics.Recall(thresholds= 0.5),
                         tfa.metrics.F1Score(num_classes=CLASS_NUMBER, threshold= 0.5)])
    model.summary()
    return model

In [2]:
trainFile = './data/lick.npy'
X = slicing(np.load(trainFile))
X = emdSignal(X)
print(X.shape)
y = np.array( [0, 1, 0, 0, 1, 0, 0, 1, 0, 1,
               1, 1, 0, 1, 0, 1, 1, 0, 0, 1,
               1, 1, 0, 0, 0, 1, 1, 1, 0, 1,
               0, 1, 1, 0, 1, 1, 1, 1, 0, 0,
               1, 1, 1, 1, 0, 1, 0, 0, 1, 1,
               0, 1, 0, 1, 1, 0, 1, 1, 0, 1,
               1, 0, 0, 1, 0, 1, 0, 1, 0, 0,
               1, 1, 0, 1, 1, 0, 0, 1, 1, 1,
               0])

(81, 300, 12)


In [ ]:
threeChan = X[18]
f = threeChan[:, 0].T
s = threeChan[:, 1].T
t = threeChan[:, 2].T
imf = emd.sift.sift(f, max_imfs=4, imf_opts={'sd_thresh': 0.1})
print(imf.shape)
emd.plotting.plot_imfs(imf)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=777)

In [ ]:
n += 1
print(n, n*SLIDING_STEP, n*SLIDING_STEP + WINDOW_SIZE)
px.line(X[n]).show()

In [10]:
model = buildModel(X.shape[1:])
history = model.fit(x=X_train,
                    y=y_train,
                    epochs=100,
                    validation_data=(X_test,y_test))

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_9 (Conv1D)           (None, 300, 80)           38480     
                                                                 
 batch_normalization_9 (Batc  (None, 300, 80)          1200      
 hNormalization)                                                 
                                                                 
 max_pooling1d_9 (MaxPooling  (None, 150, 80)          0         
 1D)                                                             
                                                                 
 dropout_9 (Dropout)         (None, 150, 80)           0         
                                                                 
 conv1d_10 (Conv1D)          (None, 150, 100)          400100    
                                                                 
 batch_normalization_10 (Bat  (None, 150, 100)        

In [349]:
f1 = np.array(history.history['loss']).flatten()
valf1 = np.array(history.history['val_loss']).flatten()
px.line(pd.DataFrame(np.array([f1, valf1]).T, columns=['loss', 'val_loss'])).show()